In [ ]:
import subprocess
import time
import sys
import random

def dns_lookup_nslookup(domain, nameserver):
    """
    nslookup을 사용하여 특정 네임서버에 DNS 조회를 수행합니다.
    성공 시 IP 주소 (문자열)를 반환하고, 실패 시 None을 반환합니다.
    """
    try:
        # Windows에서 nslookup 명령어 실행.  timeout 설정 (10초)
        result = subprocess.run(['nslookup', domain, nameserver], capture_output=True, text=True, timeout=10)

        if result.returncode == 0:
            # nslookup 출력에서 IP 주소를 파싱
            output_lines = result.stdout.splitlines()
            for line in output_lines:
                if "Address:" in line and nameserver not in line :  # "Address:" 라인을 찾음, nameserver는 제외
                    # ":" 뒤의 IP 주소 부분을 가져와 공백 제거 후 반환
                    return line.split(":", 1)[1].strip()
            return None  # IP 주소를 찾지 못한 경우
        else:
             # nslookup 실행 실패 (0이 아닌 종료 코드)
            print(f"nslookup failed (exit code {result.returncode}) with nameserver {nameserver}:\n{result.stderr}")
            return None
    except subprocess.TimeoutExpired:
        print(f"nslookup timed out with nameserver {nameserver}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


def main():
    """
    다양한 네임서버에 대해 5초 간격으로 timangs.shop의 DNS 조회를 반복,
    응답이 있으면 중단합니다.
    """
    domain = "timangs.shop"
    nameservers = [
        "8.8.8.8",          # Google Public DNS
        "1.1.1.1",          # Cloudflare DNS
        "168.126.63.1",    # Dyn

    ]

    while True:
        # 랜덤하게 네임서버 선택
        nameserver = random.choice(nameservers)
        print(f"Attempting DNS lookup for {domain} using nameserver {nameserver}...")

        ip_address = dns_lookup_nslookup(domain, nameserver)

        if ip_address:
            print(f"DNS lookup successful: {domain} -> {ip_address} (via {nameserver})")
            break  # 응답을 받으면 반복문 종료
        else:
            print(f"DNS lookup failed for {domain} using {nameserver}. Retrying in 3 seconds...")

        try:
            time.sleep(3)  # 2초 대기
        except KeyboardInterrupt:
            print("\nCtrl+C detected. Exiting.")
            sys.exit(0)

if __name__ == "__main__":
    main()

Attempting DNS lookup for www.timangs.shop using nameserver 168.126.63.1...
DNS lookup failed for www.timangs.shop using 168.126.63.1. Retrying in 3 seconds...
Attempting DNS lookup for www.timangs.shop using nameserver 168.126.63.1...
DNS lookup failed for www.timangs.shop using 168.126.63.1. Retrying in 3 seconds...
Attempting DNS lookup for www.timangs.shop using nameserver 168.126.63.1...
DNS lookup failed for www.timangs.shop using 168.126.63.1. Retrying in 3 seconds...
Attempting DNS lookup for www.timangs.shop using nameserver 1.1.1.1...
DNS lookup failed for www.timangs.shop using 1.1.1.1. Retrying in 3 seconds...
Attempting DNS lookup for www.timangs.shop using nameserver 8.8.8.8...
DNS lookup failed for www.timangs.shop using 8.8.8.8. Retrying in 3 seconds...
Attempting DNS lookup for www.timangs.shop using nameserver 8.8.8.8...
DNS lookup failed for www.timangs.shop using 8.8.8.8. Retrying in 3 seconds...
Attempting DNS lookup for www.timangs.shop using nameserver 168.126.63